# 1.- Inspect 2MASS output Catalogues

In [1]:
import numpy as np
import warnings
import os, glob, getpass, sys

from astropy.table            import Table, join, vstack, hstack, Column, MaskedColumn, unique
from astropy.utils.exceptions import AstropyWarning
from astropy                  import units as u

user = getpass.getuser()
sys.path.append('/Users/hcanovas/Astrofisica/papers/2019_clustering_oph_final/')
from extra_codes import sample_initial as samp_ini

### READ Original & 2MASS-CROSSED CATALOGUES

In [2]:
# Read Original & 2MASS Crossed catalogues =============
warnings.filterwarnings('ignore', category=AstropyWarning, append=True)

cat_0, cat_0_t = samp_ini.read_cats(inp_cat='2008_wilking_simbad.xml', inp_cat_2mass='2008_wilking_simbad_2mass.vot')
cat_1, cat_1_t = samp_ini.read_cats(inp_cat='cat_erickson_1.vot',      inp_cat_2mass='cat_erickson_1_tmasss.vot')
cat_2, cat_2_t = samp_ini.read_cats(inp_cat='cat_erickson_2.vot',      inp_cat_2mass='cat_erickson_2_tmasss.vot')
cat_3, cat_3_t = samp_ini.read_cats(inp_cat='cat_erickson_3.vot',      inp_cat_2mass='cat_erickson_3_tmasss.vot')
cat_4, cat_4_t = samp_ini.read_cats(inp_cat='2015_dunham_OPH_YSO.vot', inp_cat_2mass='2015_dunham_OPH_YSO_tmasss.vot')

# Rename cats for later ================================
cat_2008 = cat_0_t
cat_2011 = vstack([cat_1_t, cat_2_t, cat_3_t])
cat_2015 = cat_4_t

Input Targets in 2MASS Catalogue:        314
Missing 2MASS targets:                     0

Input Targets in 2MASS Catalogue:         74
Missing 2MASS targets:                     1

Input Targets in 2MASS Catalogue:         44
Missing 2MASS targets:                     0

Input Targets in 2MASS Catalogue:         17
Missing 2MASS targets:                     0

Input Targets in 2MASS Catalogue:        292
Missing 2MASS targets:                    10



# 2.- Construct Gaia Dr2 Input Table 

In [3]:
# Construct Initial Sample =============================
cat_ini = vstack([cat_2008, cat_2011, cat_2015])
cat_ini = unique(cat_ini, keys='_2MASS')
cat_ini = cat_ini['_2MASS', 'Jmag']

text    = f'TOTAL SOURCES AFTER REMOVING DUPLICATES: {len(cat_ini)}'
print('=' * len(text))
print(text)
print('=' * len(text))

TOTAL SOURCES AFTER REMOVING DUPLICATES: 465


In [4]:
# Include Source Label =================================
cat_ini['ref_1'] = ['N'] * len(cat_ini)
cat_ini['ref_2'] = ['N'] * len(cat_ini)
cat_ini['ref_3'] = ['N'] * len(cat_ini)


# Write labels =========================================
for i in range(len(cat_ini)):
    tmass = cat_ini['_2MASS'][i]
    if tmass in cat_2008['_2MASS']: cat_ini['ref_1'][i] = 'Y' # Wilking 2008
    if tmass in cat_2011['_2MASS']: cat_ini['ref_2'][i] = 'Y' # Erickson 2011
    if tmass in cat_2015['_2MASS']: cat_ini['ref_3'][i] = 'Y' # Dunham 2015


cat_ini['refs'] = [cat_ini['ref_1'][i] + cat_ini['ref_2'][i] + cat_ini['ref_3'][i] for i in range(len(cat_ini))]
set(cat_ini['refs'])

{'NNY', 'NYN', 'NYY', 'YNN', 'YNY', 'YYN', 'YYY'}

In [5]:
# Rewrite Reference label ==============================
# To check all permutations: set(cat_ini['refs'])
cat_ini['refs_2'] = ['1, 2, 3'] * len(cat_ini)

for i in range(len(cat_ini)):
    label = cat_ini['refs'][i]

    if label == 'NNY': label_2 = '3'
    if label == 'NYN': label_2 = '2'
    if label == 'NYY': label_2 = '2, 3'
    if label == 'YNN': label_2 = '1'
    if label == 'YNY': label_2 = '1, 3'
    if label == 'YYN': label_2 = '1, 2'
    if label == 'YYY': label_2 = '1, 2, 3'

    cat_ini['refs_2'][i] = label_2

In [6]:
# Write & Remove unwanted cols =========================
cat_ini = cat_ini['_2MASS', 'Jmag', 'refs_2']
cat_ini.rename_column('_2MASS', 'col2mass') # For Gaia Archive
cat_ini.rename_column('Jmag', 'jmag')       # For Gaia Archive
cat_ini.write('sample_ini.vot', format = 'votable', overwrite = True)
cat_ini[75:80]

col2mass,jmag,refs_2
,mag,
str17,float32,str7
16253812-2422362,12.838,"1, 3"
16253943-2326419,10.962,3
16253958-2426349,11.893,"1, 2, 3"
16254129-2421366,13.213,1
16254289-2325260,10.695,3


In [7]:
# Sanity Check: There are no duplicate 2MASS IDs =======
len(cat_ini), len(unique(cat_ini, 'col2mass'))

(465, 465)